In [24]:
import pandas as pd
import tensorflow as tf
import random as random
import numpy as np
#data_o = pd.read_csv("data_o.csv")
tracks = pd.read_csv("tracks.csv")

In [2]:
def rough_clean(data):
    out = data.dropna()    
    out = out.drop(["artists","name","release_date","id_artists","id"],axis=1)
    #out = out.describe(include="all")
    out["loudness"]=-out["loudness"]
    for col in out:
        out[col]=out[col]/out[col].max()
    #out["duration_ms"]=out["duration_ms"]/out["duration_ms"].max()
    
    return out
X_train =rough_clean(tracks)
#X_train=X_train.merge(tracks,on="id")
print(X_train.head())
#print(X_train.head())
#print(X_train.describe(include="all"))


   popularity  duration_ms  explicit  danceability  energy       key  \
0        0.06     0.022576       0.0      0.650858  0.4450  0.000000   
1        0.00     0.017470       0.0      0.701312  0.2630  0.000000   
2        0.00     0.032313       0.0      0.437941  0.1770  0.090909   
3        0.00     0.031471       0.0      0.323915  0.0946  0.636364   
4        0.00     0.029012       0.0      0.405651  0.1580  0.272727   

   loudness  mode  speechiness  acousticness  instrumentalness  liveness  \
0  0.222300   1.0     0.464470      0.676707            0.7440     0.151   
1  0.368933   1.0     0.985582      0.800201            0.0000     0.148   
2  0.353000   1.0     0.052729      0.997992            0.0218     0.212   
3  0.466017   1.0     0.051905      0.998996            0.9180     0.104   
4  0.281667   0.0     0.040165      0.992972            0.1300     0.311   

   valence     tempo  time_signature  
0    0.127  0.425564             0.6  
1    0.655  0.414029            

In [3]:
latent_size=7
#autoencoder
n = tf.keras.layers.Input((15))
h1 = tf.keras.layers.Dense(64)(n)
h2 = tf.keras.layers.Dense(128)(h1)
h3 = tf.keras.layers.Dense(64)(h2)
latent = tf.keras.layers.Dense(latent_size)(h3)
h4 = tf.keras.layers.Dense(64)(latent)
h5 = tf.keras.layers.Dense(128)(h4)
h6 = tf.keras.layers.Dense(64)(h5)
o = tf.keras.layers.Dense(15)(h6)
auto_enc = tf.keras.Model(inputs=n,outputs =o)
auto_enc.compile(optimizer=tf.keras.optimizers.Adam(),loss= tf.keras.losses.MSE)

In [4]:

auto_enc.fit(X_train,X_train,batch_size=256,epochs=2)

Epoch 1/2
2292/2292 [==============================] - 27s 11ms/step - loss: 0.0085
Epoch 2/2
2292/2292 [==============================] - 24s 11ms/step - loss: 0.0075


In [99]:

#Predictor

import tensorflow.keras.backend as kb
def custom_loss(y_actual,y_pred):
    custom_loss=kb.relu(y_actual-y_pred)
    return custom_loss

pred_in = tf.keras.layers.Dense(latent_size)(latent)
pred_out = tf.keras.layers.Dense(len(tracks),activation="softmax")(pred_in)
pred_model= tf.keras.Model(inputs = n, outputs = pred_out)
pred_model.compile(optimizer=tf.keras.optimizers.Adam(),loss= custom_loss)
pred_model.summary()
pred_model.layers[0].trainable=False
pred_model.layers[1].trainable=False
pred_model.layers[2].trainable=False
pred_model.layers[3].trainable=False
pred_model.layers[4].trainable=False


Model: "model_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 15)]              0         
_________________________________________________________________
dense (Dense)                (None, 64)                1024      
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 455       
_________________________________________________________________
dense_40 (Dense)             (None, 7)                 56        
_________________________________________________________________
dense_41 (Dense)             (None, 586672)            469

In [71]:
# get some labels
#picks = tracks.sample(2,axis=0).id
picks = [random.randint(0,mx) for i in range(10)]

#print (picks)
#user_data = tracks["id"].copy(deep=True)
user_data = np.array(tf.zeros(len(tracks)))

print (user_data)

print("rate these songs -1=dislike,0=undecided,1=like")

for picknum in picks:
    pick = tracks.iloc[picknum]
    
    #print(pick["name"])
    print(pick["name"], "by" ,eval( pick["artists"])[0] ,"?")    
    like = input()
    user_data[picknum]=int(like)
    #if like =="y":
        
    

[0. 0. 0. ... 0. 0. 0.]
rate these songs -1=dislike,0=undecided,1=like
How Can I Miss You When You Won't Go Away by Dan Hicks & His Hot Licks ?


 1


Party In Laguna by The Ventures ?


 -1


String Quartet No. 6 in B-Flat Major, Op. 18: I. Allegro con brio by Ludwig van Beethoven ?


...

 1


Wish I Had Not Said That by J.J. Cale ?


 0


Quiero by Dúo Hnos. Gaitán Castro ?


 -1


In [72]:
print (user_data)

[0. 0. 0. ... 0. 0. 0.]


In [73]:
def get_avg_liked_song_input(catalog_ratings):
    indxs = np.where(catalog_ratings==1)[0]
    #print(X_train.index,)
    selected = X_train.iloc[indxs,:]
    #print(selected)
    o = selected.mean().to_list()
    return o
    
X=get_avg_liked_song_input(user_data)

In [74]:
X=np.array(X).reshape(1,-1)
y = user_data.reshape(1,-1)

In [77]:
pred_model.fit(X,y,epochs=5,verbose=1)

Epoch 1/5
1/1 [==============================] - 0s 356ms/step - loss: 3.4091e-06
Epoch 2/5
1/1 [==============================] - 0s 10ms/step - loss: 3.4091e-06
Epoch 3/5
1/1 [==============================] - 0s 10ms/step - loss: 3.4091e-06
Epoch 4/5
1/1 [==============================] - 0s 11ms/step - loss: 3.4091e-06
Epoch 5/5
1/1 [==============================] - 0s 15ms/step - loss: 3.4091e-06


In [97]:
preds = pred_model.predict(X)[0]
print (preds)
top_10 = preds.argsort()[-10:][::-1]
tracks.iloc[top_10,:]
#print (np.where(preds>.0000011))


[1.7057442e-06 1.7054140e-06 1.7036723e-06 ... 1.7049464e-06 1.7041489e-06
 1.7036321e-06]


,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
368456,0PXyajDPm37BR3FKvLJvKl,Ağlamak Yok Yüreğim,48,320587,0,['Hakan Altun'],['2AhmWHekrahJri4f5e08Tf'],2000-10-12,0.314,0.4830,1,-10.597,0,0.2860,0.4720,0.000000,0.3600,0.457,51.912,5
395576,1NNoHUz33GuEWgezNnX6kv,Portami con te,40,264104,0,"[""Gigi D'Alessio""]",['4eSMsVzRJHhN1aq0IvZcyn'],1999-05-10,0.647,0.5040,11,-12.175,1,0.0424,0.1610,0.000000,0.0814,0.592,89.998,4
398573,54Jqsflm9eSeRludnzYEeq,"Ancora, Ancora, Ancora - 2001 Remaster",44,258933,0,['Mina'],['3HL1CyOnDLFJo1Rr8YBlKy'],2015-03-31,0.553,0.4020,6,-12.070,0,0.0263,0.0674,0.000791,0.3210,0.448,100.397,4
569227,0S8YdZ8DAx5ddP17iwNxRL,Dzik,38,89600,0,['Śpiewanki'],['4Y9u2QBbh6L9gzAou7Naxb'],2016-09-12,0.946,0.4950,8,-8.529,0,0.0522,0.5550,0.000951,0.0969,0.854,112.048,4
181381,6r3tsJ746xXDzVVsWahGvD,Mr. Clean,35,207667,0,['The Jam'],['2P560DaOMNDUACoH8ZhOCR'],1978-11-03,0.552,0.3980,0,-13.239,1,0.0753,0.2920,0.007620,0.1130,0.481,120.519,4
237258,5xVvPI6JVOe94OAj0xQO7g,Todo Se Pagará,40,165080,0,['Malacates Trebol Shop'],['6Nc4k4Y5rX6YgnBsoXGJjG'],2009-03-14,0.756,0.8490,5,-4.643,1,0.0319,0.1280,0.000000,0.2710,0.960,107.987,4
56453,7JJa6W4aba4AjrZ59ehfTW,Postmand Per,38,205360,0,['Postmand Per'],['6BRsQCLg2BEWf4PbBuTb6c'],1982,0.680,0.5470,5,-11.614,1,0.0450,0.6180,0.000297,0.1350,0.787,98.214,4
345278,2BwcbeUQMWtrnTyLIPUoxs,West End Blues - Live,1,256800,0,['Louis Armstrong'],['19eLuQmk9aCobbVDHc6eek'],1955,0.389,0.4430,3,-12.412,1,0.3610,0.8300,0.051600,0.8930,0.436,70.965,4
84406,1OzDyg6a1lsukiBwArG7zI,Para Empezar,66,258733,0,['Leonel García'],['3t7UqWteBBmHXkcVhMSyay'],2011-01-25,0.624,0.3520,0,-8.286,1,0.0296,0.7540,0.000002,0.0907,0.226,135.992,4
25416,0iWwDuyFpXxZEKVI7mOOdX,Ansoo Thi Meri Zindagi,0,208613,0,"['Amirbai Karnataki', 'Mohammed Rafi']","['025NPyHBtWC9aQM4STXI5V', '0gXDpqwYNDODn7fB0R...",1951-12-31,0.527,0.0955,8,-16.624,0,0.0337,0.9930,0.103000,0.4150,0.512,71.917,3


In [ ]:
print (preds)
